
Dhruuv agarwal 

## Denoising using DNN

For this the structure of the graph is just composed of traditional fully connnected layers, to build the Neural Network.

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Here we get the files as given in the document and get the required input matrices after stft and absolute to get the magnitude.

We then take the transpose of these matrices to match the format of Input matrix in the graph structure.


In [0]:
!pip install librosa # in colab, you’ll need to install this
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)
Sdata=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
Xdata=librosa.stft(sn, n_fft=1024, hop_length=512)
Xdata = np.abs(Xdata.T)
Sdata = np.abs(Sdata.T)

snt, srt=librosa.load('test_x_01.wav', sr=None)
Xtest=librosa.stft(snt, n_fft=1024, hop_length=512)
Xtestm = np.abs(Xtest.T)

snt2, srt2=librosa.load('test_x_02.wav', sr=None)
Xtest2=librosa.stft(snt2, n_fft=1024, hop_length=512)
Xtestm2 = np.abs(Xtest2.T)

print(Xdata.shape)

(2459, 513)


In [0]:
from sklearn.model_selection import train_test_split

#Xdata,Xval,Sdata,Sval = train_test_split(Xdata,Sdata,test_size=0.2)

Below are the initial default parameters defined which will be changed later.

In [0]:
n_hiddenlayer = 5
n_hiddenunits = 1024
n_classes = 10
n_inpsize = 784
batch_size = 200

Xinp = tf.placeholder('float',[None,n_inpsize])
y = tf.placeholder('float')

sizes = [n_inpsize]
[sizes.append(n_hiddenunits) for i in range(n_hiddenlayer)]
sizes.append(n_classes)

hiddenlayers = [[] for i in range(n_hiddenlayer+1)]
last = []
layers = [0.0 for i in range(n_hiddenlayer)]

Below is the function to change the parameters

In [0]:
def setparam(n_hidden,units,opclasses,inpdim,batchs):
  global n_hiddenlayer
  global n_hiddenunits
  global n_classes
  global n_inpsize
  global batch_size
  global sizes
  global layers
  global hiddenlayers
  global Xinp
  global y
  
  n_hiddenlayer = n_hidden
  n_hiddenunits = units
  n_classes = opclasses
  n_inpsize = inpdim
  batch_size = batchs

  Xinp = tf.placeholder('float',[None,n_inpsize])
  y = tf.placeholder('float',[None,n_inpsize])

  sizes = [n_inpsize]
  [sizes.append(n_hiddenunits) for i in range(n_hiddenlayer)]
  sizes.append(n_classes)

  hiddenlayers = [[] for i in range(n_hiddenlayer+1)]
  last = []
  layers = [0.0 for i in range(n_hiddenlayer)]

Graph definition as in last notebook for the previous task.

In [0]:
def NN_graph(data):
  initializer = tf.contrib.layers.xavier_initializer()
  #initializer = tf.contrib.layers.variance_scaling_initializer()
  for i in range(n_hiddenlayer+1):
    hiddenlayers[i] = {'weights': tf.Variable(initializer([sizes[i],sizes[i+1]])),
    'biases' :tf.Variable(initializer([sizes[i+1]])) }
  
  
  layers.insert(0,data)
  for i in range(0,n_hiddenlayer):
    tmp = tf.nn.relu(tf.add(tf.matmul(layers[i], hiddenlayers[i]['weights']), hiddenlayers[i]['biases']))
    layers[i+1] = tmp#tf.get_variable("var"+str(i),initializer=tf.nn.relu(tf.add(tf.matmul(layers[i], hiddenlayers[i]['weights']), hiddenlayers[i]['biases'])))
  print(layers)
  return tf.nn.relu(tf.add(tf.matmul(layers[-1], hiddenlayers[-1]['weights']), hiddenlayers[-1]['biases']))  
  
  

The actual training function, note given the small dataset and that our present machines are more than capable to handle it, batching was not found necessary in this problem.

In [0]:
output = []
output2=[]
lastlayer = []
layerdata = []
mseval=[]
def train_nn(X):
  global output
  global output2
  global lastlayer
  global layerdata
  global mseval
  pred = NN_graph(X)
  #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))
  mse = tf.losses.mean_squared_error(labels=y, predictions = pred)
  optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mse)
  
  n_epochs = 1000
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
      loss = 0
      _,c = sess.run([optimizer,mse],feed_dict={Xinp:Xdata,y:Sdata})
      loss += c
      print('Epoch',epoch,' out of ',n_epochs,' loss:',loss)
    #mseval = sess.run(mse,feed_dict = {Xinp:Xval,y:Sval})
    output = sess.run(pred,feed_dict = {Xinp:Xtestm})
    output2 = sess.run(pred,feed_dict = {Xinp:Xtestm2})
    
        

In [0]:
setparam(2,1024,513,513,100)
train_nn(Xinp)
#print(mseval)
print(output2.shape)
#print(len(lastlayer))


[<tf.Tensor 'Placeholder_258:0' shape=(?, 513) dtype=float32>, <tf.Tensor 'Relu_200:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'Relu_201:0' shape=(?, 1024) dtype=float32>]
Epoch 0  out of  1000  loss: 0.09751223027706146
Epoch 1  out of  1000  loss: 0.09427434951066971
Epoch 2  out of  1000  loss: 0.09118163585662842
Epoch 3  out of  1000  loss: 0.08826841413974762
Epoch 4  out of  1000  loss: 0.08554301410913467
Epoch 5  out of  1000  loss: 0.0829789862036705
Epoch 6  out of  1000  loss: 0.08055426180362701
Epoch 7  out of  1000  loss: 0.07825032621622086
Epoch 8  out of  1000  loss: 0.07606254518032074
Epoch 9  out of  1000  loss: 0.07398553937673569
Epoch 10  out of  1000  loss: 0.07200673967599869
Epoch 11  out of  1000  loss: 0.07012803107500076
Epoch 12  out of  1000  loss: 0.06834903359413147
Epoch 13  out of  1000  loss: 0.06666570156812668
Epoch 14  out of  1000  loss: 0.06507430970668793
Epoch 15  out of  1000  loss: 0.06356659531593323
Epoch 16  out of  1000  loss: 0.0621

We see the loss ends up to be 0.001 approx. This loss was chosen to be mse as this isnt a classification problem as last time. 

Also as we needed positive values at the end we have chosen relu activation at the last layer. 
The structure is 2 layer network with 1024 units in each layer. And the the leanrning rate was selected to be 0.0001. 

After this we get the Matrices learnt from the trained model and the Test input data, and convert them to get back the phase information as below.

This enables us to use istft and get the reconstructed version of the sound signal.

In [0]:
shtest1 = (Xtest/Xtestm.T)*output.T
shtest2 = (Xtest2/Xtestm2.T)*output2.T
print(shtest1.shape)

(513, 142)


In [0]:
#snt, srt=librosa.load('test_x_01.wav', sr=None)
#snt2, srt2=librosa.load('test_x_02.wav', sr=None)

#Xtest=librosa.stft(snt, n_fft=1024, hop_length=512)
iStftMat1 = librosa.istft(shtest1, hop_length=512)
iStftMat2 = librosa.istft(shtest2, hop_length=512)


librosa.output.write_wav('test_s_01_recons.wav', iStftMat1, srt)
librosa.output.write_wav('test_s_02_recons.wav', iStftMat2, srt2)